In [12]:
#inititalize the notebook
import os
import textwrap
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

chat_deployment_name = "gpt-4o"

print("*** init done! ***")


*** init done! ***


simple prompt

In [13]:
question = "Wat voor school is het Sint-Janscollege in Hoensbroek?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

Het Sint-Janscollege in Hoensbroek is een middelbare school in Nederland. De
school biedt onderwijs aan op verschillende niveaus, waaronder vmbo, havo en
vwo. Het Sint-Janscollege is onderdeel van de Stichting Limburgs Voortgezet
Onderwijs (LVO), die meerdere scholen in de regio beheert. De school legt de
nadruk op een veilige en inspirerende leeromgeving, en biedt diverse
faciliteiten en mogelijkheden voor persoonlijke ontwikkeling en buitenschoolse
activiteiten.


system prompt

In [14]:
question = "Wat voor school is het Sint-Janscollege in Hoensbroek?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent die antwoord als een grappige kat"
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

Miauw! Sint-Janscollege in Hoensbroek is een middelbare school, oftewel een plek
waar mensenjongens en -meisjes zichzelf slimmer maken. Ze hebben daar van alles,
van brugklas tot eindexamen, zodat je brein net zo groot wordt als een flinke
doos kattenvoer. Dus, of je nou een vwo-poes of mavo-kater bent, ze hebben er
voor ieder wat wils! Miauw! 🐾🎓


RESET THE SYSTEM PROMPT

In [15]:
firstQuestion = "Wat voor school is het Sint-Janscollege in Hoensbroek?"

firstResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": firstQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(firstResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Het Sint-Janscollege in Hoensbroek is een middelbare school. Het biedt
verschillende onderwijsvormen aan, zoals vmbo, havo en vwo. De school richt zich
op een brede ontwikkeling van leerlingen en streeft naar een stimulerende
leeromgeving. Daarnaast is er aandacht voor persoonlijke begeleiding en
buitenschoolse activiteiten.


vervolgvraag

In [ ]:
secondQuestion = "Wat is het adres?"

secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Waarvoor heb je een adres nodig of over welk adres heb je vragen?


Add more context

In [31]:
systemWithMemory = {
    "role": "system",
    "content": f"""
            use the previous question and answer as a context 
            ##
            previous question {firstQuestion}
            previous answer {firstResponse.choices[0].message.content}
            """
}


secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        },
        systemWithMemory
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Het adres van het Sint-Janscollege in Hoensbroek is:  Sint-Janscollege Randweg 1
6431 DD Hoensbroek, Nederland


short term memory - langchain

In [32]:
from langchain_community.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

load_dotenv()


# LLM
llm = AzureChatOpenAI(
    deployment_name=chat_deployment_name,
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",
) 


# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Je bent een behulpzame assistent"
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=False, memory=memory)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
print("---------")
print(firstQuestion)
print("---------")
response = conversation({"question": firstQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)


print("---------")
print(secondQuestion + " (nu hebben we het antwoord van de eerste vraag als context)")
print("---------")
response = conversation({"question": secondQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)




---------
Wat voor school is het Sint-Janscollege in Hoensbroek?
---------
Het Sint-Janscollege in Hoensbroek is een middelbare school die verschillende
onderwijsvormen aanbiedt, zoals vmbo, havo en vwo. De school legt de nadruk op
een brede ontwikkeling van leerlingen en biedt diverse extra-curriculaire
activiteiten en ondersteuning aan. Het is een school die zich richt op zowel
academische prestaties als persoonlijke groei.
---------
Wat is het adres? (nu hebben we het antwoord van de eerste vraag als context)
---------
Het adres van het Sint-Janscollege in Hoensbroek is Maartensdijk 2, 6432 GP
Hoensbroek, Nederland.


Je kunt nu gewoon verder chatten, memory wordt steeds verder uitgebreid

Let op! Je prompt wordt steeds groter, dus je verbruikt meer tokens
Wellicht heb je niet alle historie van de chat nodig.

dit is ook een voorbeeld hoe je een specifieke instructie geeft. "antwoord alleen met de link naar Google Maps" is een hele specifieke instructie.

In [34]:
thirdQuestion = """
                    hoe zou een link naar Google Maps eruit zien voor dit adres?
                    antwoord alleen met de link naar Google Maps op 1 regel
                """
print("---------")
print(thirdQuestion)
print("---------")
response = conversation({"question": thirdQuestion})

lines = textwrap.wrap(response["text"], width=160)
for line in lines:
    print(line)



---------

                    hoe zou een link naar Google Maps eruit zien voor dit adres?
                    antwoord alleen met de link naar Google Maps op 1 regel
                
---------
https://www.google.com/maps?q=Maartensdijk+2,+6432+GP+Hoensbroek,+Nederland


nog een voorbeeld van een hele specifieke instructie

In [35]:
JsonQuestion = """
                    format your output in a json format
                    output a plain json as provided in the example with the address
                    ## sample json
                    {
                        "street": "street",
                        "city": "city",
                        "country": "country",
                        "postal_code": "postal_code"
                    }
                                        
                """
print("---------")
print(JsonQuestion)
print("---------")
response = conversation({"question": JsonQuestion})

print(response["text"])

---------

                    format your output in a json format
                    output a plain json as provided in the example with the address
                    ## sample json
                    {
                        "street": "street",
                        "city": "city",
                        "country": "country",
                        "postal_code": "postal_code"
                    }
                                        
                
---------
```json
{
    "street": "Maartensdijk 2",
    "city": "Hoensbroek",
    "country": "Nederland",
    "postal_code": "6432 GP"
}
```


Een hele specifieke vraag

In [36]:
mySpecificQuestion = "wanneer is de informatieavond?"

mySpecificResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": mySpecificQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)


lines = textwrap.wrap(mySpecificResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)



Voor welke informatieavond heb je interesse? Als je meer details kunt geven,
zoals de school, organisatie of het onderwerp, kan ik je beter helpen.


Wellicht kun je wat meer context geven in de prompt.

We gaan nu data toevoegen, hele specifieke data, namelijk een aantal pagina's
van de Sintjan Website.

In [38]:
#lees de pagina's om deze later toe te voegen in de prompt
page1 = open("../pages/home.md", "r").read()
page2 = open("../pages/informatieavond-5-november.md", "r").read()
pages = [
    {
        "fileName": "home.md",
        "content": page1
    },
    {
        "fileName": "informatieavond-5-november.md",
        "content": page2
    }
   
]
    

myDataQuestion = "wanneer is de informatieavond?"
try:
    myDataResponse = openai_client.chat.completions.create(
        model = chat_deployment_name,
        temperature=0.0,
        messages=[
            {
                "role": "user",
                "content": myDataQuestion
            },
            {
                "role": "system",
                "content" : f"""
                    Je bent een behulpzame assistent
                    Je geeft alleen antwoord op basis van de data die je hebt gekregen
                    Indien je het antwoord niet kunt vinden, zeg je dat je het antwoord niet weet
                    vermeld je bron document bij je antwoord

                    ## data
                    {pages}

                    """
            }
        ]
    )

    lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
    for line in lines:
        print(line)

    print("----")
    print("Hier wat info over de tokens die je hebt gebruikt:")
    print(myDataResponse.usage)
except Exception as e:
    print(e)

Het adres van de school is:  Amstenraderweg 122   6431 EN Hoensbroek  Bron:
home.md
----
Hier wat info over de tokens die je hebt gebruikt:
CompletionUsage(completion_tokens=26, prompt_tokens=6112, total_tokens=6138, completion_tokens_details=None, prompt_tokens_details=None)


Okay....dit lijkt werken, maar we verbruiken heel veel tokens...tijd voor een volgende stap.

***Embeddings en vectoren